### Trial 1: Markov model

In [17]:
import numpy as np
import string

np.random.seed(1234)#to get consistent results

In [18]:
initial = {} 
first_order = {} 
second_order = {} 

In [19]:
# def remove_punctuation(s):
#     return s.translate(str.maketrans('','',string.punctuation))
def remove_punctuation(s):
    return s

In [20]:
def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [21]:
for line in open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt"):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T - 1:

                add2dict(second_order, (t_1, t), 'END')
            if i == 1:

                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [22]:

initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [23]:
def list2pdict(ts):
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c / n
    return d

In [24]:
for t_1, ts in first_order.items():
    first_order[t_1] = list2pdict(ts)

In [25]:
for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

In [26]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False)

In [27]:
def generate():
    for i in range(6): 
        sentence = []

        w0 = sample_word(initial)
        sentence.append(w0)

        w1 = sample_word(first_order[w0])
        sentence.append(w1)

        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [28]:
generate()

i'm so accustomed to the heart that broke so long—
then thought of us, and return—
those boys and girls
but internal difference
eden—a legend—dimly told—
or what circassian land?


### Trial 2: LSTM

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [34]:
with open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt", 'r', encoding='utf-8') as file:
    text = file.read()

text = text.lower()  
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(predictors, label, epochs=100, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 100)           1325500   
                                                                 
 lstm (LSTM)                 (None, 11, 150)           150600    
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 13255)             1338755   
                                                                 
Total params: 2,915,255
Trainable params: 2,915,255
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1554/1554 [==============================] - 69s 43ms/step - loss: 7.8790 - accuracy: 0.0618
Epoch 2/100
1554/1554 [==============================] - 64s 41ms/step - 

In [36]:
seed_text = "I am"
next_words = 20  

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 23ms/step
I am alive—because bold so far— suspect me with him the sea of that— by steel he not be ended— – along


### Implementing Bidirectional LSTM

In [66]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional

In [67]:
with open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt", 'r', encoding='utf-8') as file:
    text = file.read()

text = text.lower()  
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(predictors, label, epochs=10, verbose=1)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 11, 100)           1325500   
                                                                 
 lstm_15 (LSTM)              (None, 11, 150)           150600    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dense_11 (Dense)            (None, 13255)             2664255   
                                                                 
Total params: 4,341,155
Trainable params: 4,341,155
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1554/1554 [==============================] - 97s 60ms/step - loss: 7.8496 - accuracy: 0.0620
Epoch

In [75]:
seed_text = "Summer is"
next_words = 20  

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 [==============================] - 0s 18ms/step


In [76]:
print(seed_text)

Summer is the sun of the sun of the sun air come— swain i come— come— swain swain swain one santa workman—


In [70]:
# Generate poetry in multiple lines
seed_text = "Summer is"
next_lines = 5 
words_per_line = 5  
for _ in range(next_lines):
    generated_words = []
    for _ in range(words_per_line):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        generated_words.append(output_word)
    generated_line = " ".join(generated_words)
    print(generated_line)


1/1 [==============================] - 0s 17ms/step
the sun of the sun
1/1 [==============================] - 0s 19ms/step
of the sun air come—
1/1 [==============================] - 0s 19ms/step
swain i come— come— swain
1/1 [==============================] - 0s 19ms/step
swain swain one santa workman—
1/1 [==============================] - 0s 17ms/step
workman— santa santa workman— workman—


### Bidirectional LSTM without punctuations

In [64]:
import numpy as np
import tensorflow as tf
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional

# Read the text file
with open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt", 'r', encoding='utf-8') as file:
    text = file.read()

# Convert text to lowercase
text = text.lower()

# Remove punctuation
text = text.translate(str.maketrans("", "", string.punctuation))

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(predictors, label, epochs=10, verbose=1)

seed_text = "I am"
next_words = 20  

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

    if output_word == '\n':
        print(seed_text.strip())
        seed_text = ""

print(seed_text.strip())


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 11, 100)           1321000   
                                                                 
 lstm_13 (LSTM)              (None, 11, 150)           150600    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              200800    
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 13210)             2655210   
                                                                 
Total params: 4,327,610
Trainable params: 4,327,610
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 258/1547 [====>.........................] - ETA: 1:17 - loss: 8.1571 - accuracy: 0.0624

KeyboardInterrupt: 

### With attention layer

In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, LSTM, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical



"Poems\Emily Dickinson\Emily Dickinsons' poems.txt"

In [46]:
class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, inputs):
        query, value = inputs
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(value)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * value
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

with open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt", 'r') as file:
    poems_text = file.read()

poems_text = poems_text.lower()
poems_text = poems_text.replace('\n', ' \n ')
poems_text = poems_text.replace('\r', ' ')
poems_text = ' '.join(poems_text.split())

# Creating the vocabulary
words = poems_text.split()
word_to_index = {word: i for i, word in enumerate(set(words))}
index_to_word = {i: word for word, i in word_to_index.items()}
vocab_size = len(word_to_index)

# Generate input-output pairs
sequences = []
next_words = []
sequence_length = 10

for i in range(len(words) - sequence_length):
    sequence = words[i:i+sequence_length]
    target = words[i+sequence_length]
    sequences.append([word_to_index[word] for word in sequence])
    next_words.append(word_to_index[target])

sequences = np.array(sequences)
next_words = np.array(next_words)

embedding_dim = 100
hidden_units = 256

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=sequence_length))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(BahdanauAttention(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(sequences, next_words, epochs=10, batch_size=32)

OperatorNotAllowedInGraphError: Exception encountered when calling layer "bahdanau_attention_1" (type BahdanauAttention).

in user code:

    File "C:\Users\lmbmo\AppData\Local\Temp\ipykernel_7876\4115574913.py", line 10, in call  *
        query, value = inputs

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.


Call arguments received by layer "bahdanau_attention_1" (type BahdanauAttention):
  • inputs=tf.Tensor(shape=(None, 10, 256), dtype=float32)

In [ ]:
start_sequence = "i"
generated_poem = [start_sequence]
num_lines = 10

for _ in range(num_lines):
    encoded_input = [word_to_index[word] for word in generated_poem]
    encoded_input = pad_sequences([encoded_input], maxlen=sequence_length)
    predicted_index = np.argmax(model.predict(encoded_input))
    predicted_word = index_to_word[predicted_index]
    generated_poem.append(predicted_word)

for line in generated_poem:
    print(line)


In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re     
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [ ]:
!

In [1]:
# -*- coding: utf-8 -*-
"""Poem text generator.ipynb

Automatically generated by Colaboratory.

# Text Generation using LSTM

##Two types of text generation


1.   Character based text generation
* Each character of the text is used to train the model and prediction will also result in generation of new characters. 

2.   Word based text generation
* Words are converted into tokens which are used to train the model. The model will generate words instead of characters in the prediction stage.

## Mechanics of the text generation model: 

1. The next word of the sequence is predicted using the words that are already present in the sequence. 

2. It is a simple model where splitting the data into training and testing sets is not required. This is because the model will use all the words in the sequence to predict the next word. Just like forecasting.

#The flow of program:

1. Loading data
2. Preprocessing the data and Tokenizing 
3. Building and fitting the model on data
4. Evaluate the model
5. Predicting(Generating the text)
6. Saving the model for future applicaitons
"""

# # mounting the drive to read the data
# from google.colab import drive
# drive.mount('/content/drive/')



# reading the data from local google drive 
with open("Poems\Emily Dickinson\Emily Dickinsons' poems.txt") as story:
  story_data = story.read()


In [9]:
def clean_text(text):
    text = re.sub(r',', '', text)
    text = re.sub(r'\'', '',  text)
    text = re.sub(r'\"', '', text)
    text = re.sub(r'\(', '', text)
    text = re.sub(r'\)', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'“', '', text)
    text = re.sub(r'”', '', text)
    text = re.sub(r'’', '', text)
    text = re.sub(r'\.', '', text)
    text = re.sub(r';', '', text)
    text = re.sub(r':', '', text)
    text = re.sub(r'\-', '', text)

    return text

# cleaning the data
lower_data = story_data.lower()           # Converting the string to lower case to get uniformity

split_data = lower_data.splitlines()      # Splitting the data to get every line seperately but this will give the list of uncleaned data

print(split_data)                         

final = ''                                # initiating a argument with blank string to hold the values of final cleaned data

for line in split_data:
  line = clean_text(line)
  final += '\n' + line

# print(final)

final_data = final.split('\n')       # splitting again to get list of cleaned and splitted data ready to be processed
# print(final_data)

['', 'as if i asked a common alms,', '', 'and in my wondering hand', '', 'a stranger pressed a kingdom,', '', 'and i, bewildered, stand—', '', 'as if i asked the orient', '', 'had it for me a morn—', '', 'and it should lift its purple dikes,', '', 'and shatter me with dawn!', '', 'it always felt to me—a wrong', '', 'to that old moses—done—', '', 'to let him see—the canaan—', '', 'without the entering—', '', '', "and tho' in soberer moments—", '', 'no moses there can be', '', "i'm satisfied—the romance", '', 'in point of injury—', '', '', 'surpasses sharper stated—', '', 'of stephen—or of paul—', '', 'for these—were only put to death—', '', "while god's adroiter will", '', '', 'on moses—seemed to fasten', '', 'with tantalizing play', '', 'as boy—should deal with lesser boy—', '', 'to prove ability.', '', '', "the fault—was doubtless israel's—", '', 'myself—had banned the tribes—', '', 'and ushered grand old moses', '', 'in pentateuchal robes', '', '', 'upon the broad possession', '', "'

In [10]:

# Instantiating the Tokenizer
max_vocab = 1000000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(final_data)

# Getting the total number of words of the data.
word2idx = tokenizer.word_index
print(len(word2idx))
print(word2idx)
vocab_size = len(word2idx) + 1        # Adding 1 to the vocab_size because the index starts from 1 not 0. This will make it uniform when using it further
print(vocab_size)

"""## Creating n-gram sequences from the sentences

* Consider this sentence : ['two roads diverged in a yellow wood']. Here we will use ['two roads diverged in a yellow'] to predict ['wood']. This is the basic concept of forecasting which can be applied here to generate text.

* An advacement of this will be to use single word or every combination words possible from the sentence to predict the next word. And this is loosely termed as n_gram sequences

* The sentence ['two roads diverged in a yellow wood'] will have sequence as [112, 113, 114, 7, 5, 190, 75]

* so we will use combinations of words to make our model better

* [112, 113], 
* [112, 113, 114], 
* [112, 113, 114, 7], 
* [112, 113, 114, 7, 5], 
* [112, 113, 114, 7, 5, 190], 
* [112, 113, 114, 7, 5, 190, 75]

* we train our model that if 112 comes then it has to predict 113.
* if combination of 112, 113, comes then it has to predict 114 and so on.

"""

13208
{'the': 1, 'a': 2, 'and': 3, 'to': 4, 'i': 5, 'of': 6, 'in': 7, 'it': 8, 'that': 9, 'as': 10, 'my': 11, 'is': 12, 'for': 13, 'not': 14, 'but': 15, 'so': 16, 'be': 17, 'her': 18, 'with': 19, 'when': 20, 'his': 21, 'me': 22, 'if': 23, 'was': 24, 'at': 25, 'from': 26, 'we': 27, 'or': 28, 'he': 29, 'no': 30, 'by': 31, 'on': 32, 'like': 33, 'you': 34, 'this': 35, 'an': 36, 'upon': 37, 'all': 38, 'one': 39, 'they': 40, 'could': 41, 'its': 42, 'then': 43, 'had': 44, 'how': 45, 'just': 46, 'little': 47, 'would': 48, 'were': 49, 'what': 50, 'their': 51, 'have': 52, 'are': 53, 'our': 54, 'she': 55, 'too': 56, 'will': 57, 'me—': 58, 'than': 59, 'who': 60, 'know': 61, 'never': 62, 'day': 63, 'there': 64, 'some': 65, 'away': 66, 'such': 67, 'till': 68, 'where': 69, 'can': 70, 'tis': 71, 'should': 72, 'him': 73, 'your': 74, 'did': 75, 'yet': 76, 'do': 77, 'nor': 78, 'tell': 79, 'thee': 80, 'other': 81, 'go': 82, 'more': 83, 'may': 84, 'life': 85, 'twas': 86, 'only': 87, 'cannot': 88, 'has': 89

"## Creating n-gram sequences from the sentences\n\n* Consider this sentence : ['two roads diverged in a yellow wood']. Here we will use ['two roads diverged in a yellow'] to predict ['wood']. This is the basic concept of forecasting which can be applied here to generate text.\n\n* An advacement of this will be to use single word or every combination words possible from the sentence to predict the next word. And this is loosely termed as n_gram sequences\n\n* The sentence ['two roads diverged in a yellow wood'] will have sequence as [112, 113, 114, 7, 5, 190, 75]\n\n* so we will use combinations of words to make our model better\n\n* [112, 113], \n* [112, 113, 114], \n* [112, 113, 114, 7], \n* [112, 113, 114, 7, 5], \n* [112, 113, 114, 7, 5, 190], \n* [112, 113, 114, 7, 5, 190, 75]\n\n* we train our model that if 112 comes then it has to predict 113.\n* if combination of 112, 113, comes then it has to predict 114 and so on.\n\n"

In [11]:
# We will turn the sentences to sequences line by line and create n_gram sequences

input_seq = []

for line in final_data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    input_seq.append(n_gram_seq)

print(input_seq)

# Getting the maximum length of sequence for padding purpose
max_seq_length = max(len(x) for x in input_seq)
print(max_seq_length)

# Padding the sequences and converting them to array
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_length, padding='pre'))
print(input_seq)

# Taking xs and labels to train the model.

xs = input_seq[:, :-1]        # xs contains every word in sentence except the last one because we are using this value to predict the y value
labels = input_seq[:, -1]     # labels contains only the last word of the sentence which will help in hot encoding the y value in next step
print("xs: ",xs)
print("labels:",labels)


[[10, 23], [10, 23, 5], [10, 23, 5, 573], [10, 23, 5, 573, 2], [10, 23, 5, 573, 2, 395], [10, 23, 5, 573, 2, 395, 4388], [3, 7], [3, 7, 11], [3, 7, 11, 4389], [3, 7, 11, 4389, 173], [2, 1198], [2, 1198, 2636], [2, 1198, 2636, 2], [2, 1198, 2636, 2, 1484], [3, 5], [3, 5, 1485], [3, 5, 1485, 1199], [10, 23], [10, 23, 5], [10, 23, 5, 573], [10, 23, 5, 573, 1], [10, 23, 5, 573, 1, 2637], [44, 8], [44, 8, 13], [44, 8, 13, 22], [44, 8, 13, 22, 2], [44, 8, 13, 22, 2, 396], [3, 8], [3, 8, 72], [3, 8, 72, 329], [3, 8, 72, 329, 42], [3, 8, 72, 329, 42, 180], [3, 8, 72, 329, 42, 180, 4390], [3, 4391], [3, 4391, 22], [3, 4391, 22, 19], [3, 4391, 22, 19, 483], [8, 885], [8, 885, 249], [8, 885, 249, 4], [8, 885, 249, 4, 4392], [8, 885, 249, 4, 4392, 1900], [4, 9], [4, 9, 119], [4, 9, 119, 4393], [4, 133], [4, 133, 73], [4, 133, 73, 4394], [4, 133, 73, 4394, 4395], [94, 1], [94, 1, 4396], [3, 291], [3, 291, 7], [3, 291, 7, 4397], [3, 291, 7, 4397, 1901], [30, 1902], [30, 1902, 64], [30, 1902, 64, 70]

In [13]:
# one-hot encoding the labels according to the vocab size

# The matrix is square matrix of the size of vocab_size. Each row will denote a label and it will have 
# a single +ve value(i.e 1) for that label and other values will be zero. 

ys = to_categorical(labels, num_classes=vocab_size)
# print(ys)



# using the functional APIs of keras to define the model

i = Input(shape=(max_seq_length - 1, ))                           # using 1 less value becasuse we are preserving the last value for predicted word 
x = Embedding(vocab_size, 124)(i)
x = Dropout(0.2)(x)
x = LSTM(520, return_sequences=True)(x)
x = Bidirectional(layer=LSTM(340, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(vocab_size, activation='softmax')(x)

model = Model(i,x)


# using the pipeline method of sequential to define a model
# model = Sequential()
# model.add(Embedding(vocab_size, 124, input_length=max_seq_length-1))
# model.add(Dropout(0.2))
# model.add(LSTM(520, return_sequences=True))
# model.add(Bidirectional(LSTM(340, return_sequences=True)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(vocab_size, activation='softmax'))

# model.compile(optimizer=Adam(lr=0.001),
model.compile(optimizer=Adam(learning_rate=0.001),              
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

# model.summary()                                       
r = model.fit(xs,ys,epochs=100)

Epoch 1/100
1547/1547 [==============================] - 579s 368ms/step - loss: 7.6663 - accuracy: 0.0629
Epoch 2/100
1547/1547 [==============================] - 540s 349ms/step - loss: 7.2026 - accuracy: 0.0701
Epoch 3/100
1547/1547 [==============================] - 537s 347ms/step - loss: 6.9549 - accuracy: 0.0702
Epoch 4/100
1547/1547 [==============================] - 537s 347ms/step - loss: 6.7619 - accuracy: 0.0733
Epoch 5/100
1547/1547 [==============================] - 536s 347ms/step - loss: 6.5622 - accuracy: 0.0797
Epoch 6/100
1547/1547 [==============================] - 3626s 2s/step - loss: 6.3632 - accuracy: 0.0841
Epoch 7/100
1547/1547 [==============================] - 547s 354ms/step - loss: 6.1725 - accuracy: 0.0877
Epoch 8/100
1547/1547 [==============================] - 585s 378ms/step - loss: 6.0010 - accuracy: 0.0895
Epoch 9/100
1547/1547 [==============================] - 595s 385ms/step - loss: 5.8347 - accuracy: 0.0938
Epoch 10/100
1547/1547 [===============

In [ ]:
plt.plot(r.history['accuracy'])

In [ ]:
def predict_words(seed, no_words):
  for i in range(no_words):
    token_list = tokenizer.texts_to_sequences([seed])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=1)

    new_word = ''

    for word, index in tokenizer.word_index.items():
      if predicted == index:
        new_word = word
        break
    seed += " " + new_word
  print(seed)

In [ ]:
# predicting or generating the poem with the seed text

seed_text = 'love is'
next_words = 20

predict_words(seed_text, next_words)

In [ ]:
# saving the model

model.save('poem_generator.h5') # Will create a HDF5 file of the model